In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
response = llm.invoke("What is the capital of France?")
print(response.content)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
response = llm.invoke("What is the capital of France?")
print(response.content)

from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="You are a helpful assistant that can answer questions about the weather."),
    HumanMessage(content="What is the weather in Tokyo?")
]

response = llm.invoke(messages)
print(response.content)


The capital of France is Paris.
The capital of France is Paris.
I don't have real-time weather data, but you can easily check the current weather in Tokyo by using a weather website or app. Typically, these platforms provide up-to-date information on temperature, humidity, precipitation, and forecasts. Would you like some tips on how to find this information?
Testing calculator tool



In [25]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions. Allways use this for any math calculations."""
    try:
        result = eval(expression)
        return f"The result of the expression {expression} is {result}"
    except Exception as e:
        return f"Error calcuating {expression}: {e}"
    

search_tool = DuckDuckGoSearchRun()

# Bind tools to the LLM
tools = [calculator, search_tool]
llm_with_tools = llm.bind_tools(tools)

print("Testing calculator tool")
response = llm_with_tools.invoke("What is 25 * 4 + 17?")
print(response.content)

Testing calculator tool



In [26]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1M9wj3z9Mp3aE5FjVIKhnDIr', 'function': {'arguments': '{"expression":"25 * 4 + 17"}', 'name': 'calculator'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 110, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-ByeiLIKEPl6pXGWqp0YM1ZyhznZvP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8a5c5f80-4f13-4e76-bcd2-e12dacf80584-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '25 * 4 + 17'}, 'id': 'call_1M9wj3z9Mp3aE5FjVIKhnDIr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 110, 'output_tokens': 20, 'total_tokens': 130, 'inpu

In [27]:
tool_map = {
    "calculator": calculator,
    "duckduckgo_search": DuckDuckGoSearchRun()
}

In [28]:
def handle_tool_calls(response, tool_map):
    """Executes all tool calls in the LLM response using the tool map."""
    if not getattr(response, "tool_calls", None):
        return response
    print(f"Tool calls requested: {len(response.tool_calls)}")
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"]
        args = tool_call["args"]

        print(f"Tool: {tool_name}")
        print(f"Args: {args}")
        
        tool = tool_map.get(tool_name)
        if tool:
            result = tool.invoke(args)
            preview = result[:200] + "..." if isinstance(result, str) and len(result) > 200 else result
            print(f"Tool result: {preview}")

In [29]:
def test_llm_tool(query):
    print(f"Query: {query}")
    response = llm_with_tools.invoke(query)
    print(f"Response: {getattr(response, 'content', response)}")
    handle_tool_calls(response, tool_map)
    print("\n")


test_llm_tool("What is 25 * 4 + 17?")
test_llm_tool("Search for the latest news abount Artificial Intelligence.")

Query: What is 25 * 4 + 17?
Response: 
Tool calls requested: 1
Tool: calculator
Args: {'expression': '25 * 4 + 17'}
Tool result: The result of the expression 25 * 4 + 17 is 117


Query: Search for the latest news abount Artificial Intelligence.
Response: 
Tool calls requested: 1
Tool: duckduckgo_search
Args: {'query': 'latest news about Artificial Intelligence'}


c:\Portfolio\langgraph_exploration\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Tool result: Jun 25, 2025 · Artificial Intelligence News. Everything on AI including futuristic robots with artificial intelligence, computer models of human intelligence and more. A Taruma Baterias oferece bateri...




Structured Output

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class PersonInfo(BaseModel):
    """Information about a person."""
    name: str = Field(description="The full name of the person.")
    age: int = Field(description="The age of the person.")
    occupation: Optional[str] = Field(description="The occupation of the person.")
    skills: List[str] = Field(description="The skills of the person.")

structured_llm = llm.with_structured_output(PersonInfo)


#Test with person info
print("Testing structured output")

person_prompt = """
Extract information about the person:
"John Smith is 30 years old and works as a software engineer. He is skilled in Python, SQL, and machine learning."
"""

response = structured_llm.invoke(person_prompt)
print(response)


print("Name: ", response.name)
print("Age: ", response.age)
print("Occupation: ", response.occupation)
print("Skills: ", ", ".join(response.skills))

Testing structured output
name='John Smith' age=30 occupation='software engineer' skills=['Python', 'SQL', 'machine learning']
Name:  John Smith
